
# **TP : Prévision d’une série temporelle avec Deep Learning**



1.   Élément de liste
2.   Élément de liste



*   L'ensemble de données climatiques de Jena est composé de 14 variables différentes (telles que la température de l'air, la pression atmosphérique, l'humidité, la direction du vent,
etc.) qui ont été enregistrées toutes les 10 minutes, sur plusieurs années.
*   Ce jeu de données couvre la période du 1er janvier 2009 au 31 décembre 2016. ( disponible sur ce lien https://www.kaggle.com/datasets/mnassrib/jena-climate)
*  
Notre objectif est de prédire la série temporelle de la température (colonne intitulée T (degC)) en utilisant uniquement cette variable.

## Importation des bibliothèques

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from tensorflow import keras
from sklearn.preprocessing import StandardScaler


## Téléchargement de la base de données

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
data = pd.read_csv(r'/content/drive/MyDrive/Proj.csv')

# Traitement et Extraction des Données du Fichier Climatique

In [ ]:
# Extraction de la colonne "Date Time" pour les dates
dates = data["Date Time"]

# Extraction de la colonne "T (degC)" (température)
temperature = data["T (degC)"]

# Conversion des données restantes en un tableau (sans "Date Time")
raw_data = data.drop(columns=["Date Time"]).to_numpy()

# Affichage des premières lignes pour vérifier
#print("Premières dates :", dates.head().tolist())
#print("Premières températures :", temperature.head().tolist())
#print("Extrait des données brutes :", raw_data[:5])

In [ ]:
# Tracer la température en fonction du temps
plt.figure(figsize=(12, 5))
plt.plot(dates, temperature, label='Température (°C)')
plt.xlabel('Date')
plt.ylabel('Température (°C)')
plt.title('Température en fonction du temps')
plt.legend()

# Séparation des données en ensembles d'entraînement, de validation et de test

In [ ]:
# Séparation des données en ensembles d'entraînement, de validation et de test

# Calcul du nombre d'échantillons pour chaque ensemble
num_train_samples = int(0.7 * len(raw_data))  # 70% des données pour l'entraînement
num_val_samples = int(0.15 * len(raw_data))   # 15% des données pour la validation
num_test_samples = len(raw_data) - num_train_samples - num_val_samples  # Le reste pour le test

# Affichage du nombre d'échantillons dans chaque ensemble
print("num_train_samples:", num_train_samples)
print("num_val_samples:", num_val_samples)
print("num_test_samples:", num_test_samples)



### Preparation des données

In [ ]:
# Normalisation des Données
# Calcul de la moyenne des données d'entraînement
mean = raw_data[:num_train_samples].mean(axis=0)

# Soustraction de la moyenne à l'ensemble des données pour centrer les données
raw_data -= mean

# Calcul de l'écart type des données d'entraînement
std = raw_data[:num_train_samples].std(axis=0)

# Division par l'écart type pour normaliser les données
raw_data /= std
#print(raw_data[:1])

In [ ]:
# Création des Jeux de Données de Séries Temporelles pour l'Entraînement, la Validation et le Test

# Paramètres pour la création des séries temporelles
sampling_rate = 6  # Taux d'échantillonnage
sequence_length = 120  # Longueur de chaque séquence
delay = sampling_rate * (sequence_length + 24 - 1)  # Délai pour aligner les cibles
batch_size = 256  # Taille de chaque batch

# Création du jeu de données d'entraînement
train_dataset = keras.utils.timeseries_dataset_from_array(
    data=raw_data[:-delay],  # Données d'entrée (tout sauf les derniers 'delay' éléments)
    targets=temperature[delay:],  # Cibles (décalées de 'delay' éléments)
    sampling_rate=sampling_rate,  # Taux d'échantillonnage
    sequence_length=sequence_length,  # Longueur de chaque séquence
    shuffle=True,  # Mélanger les données pour l'entraînement
    batch_size=batch_size,  # Taille de chaque batch
    start_index=0,  # Index de début des données d'entraînement
    end_index=num_train_samples  # Index de fin des données d'entraînement
)

# Création du jeu de données de validation
val_dataset = keras.utils.timeseries_dataset_from_array(
    data=raw_data[:-delay],  # Données d'entrée (tout sauf les derniers 'delay' éléments)
    targets=temperature[delay:],  # Cibles (décalées de 'delay' éléments)
    sampling_rate=sampling_rate,  # Taux d'échantillonnage
    sequence_length=sequence_length,  # Longueur de chaque séquence
    shuffle=True,  # Mélanger les données pour la validation
    batch_size=batch_size,  # Taille de chaque batch
    start_index=num_train_samples,  # Index de début des données de validation
    end_index=num_train_samples + num_val_samples  # Index de fin des données de validation
)

# Création du jeu de données de test
test_dataset = keras.utils.timeseries_dataset_from_array(
    data=raw_data[:-delay],  # Données d'entrée (tout sauf les derniers 'delay' éléments)
    targets=temperature[delay:],  # Cibles (décalées de 'delay' éléments)
    sampling_rate=sampling_rate,  # Taux d'échantillonnage
    sequence_length=sequence_length,  # Longueur de chaque séquence
    shuffle=True,  # Mélanger les données pour le test
    batch_size=batch_size,  # Taille de chaque batch
    start_index=num_train_samples + num_val_samples  # Index de début des données de test
)

# Création des Jeux de Données de Séries Temporelles pour l'Entraînement, la Validation et le Test


**Inspecting the output of one of our datasets**

In [ ]:
# Vérification de la Forme des Échantillons et des Cibles dans le Jeu de Données d'Entraînement

# Boucle pour itérer sur le jeu de données d'entraînement
for samples, targets in train_dataset:
    # Afficher la forme des échantillons
    print("samples shape:", samples.shape)

    # Afficher la forme des cibles
    print("targets shape:", targets.shape)

    # Sortir de la boucle après la première itération
    break

# Modèle LSTM

In [ ]:
# Compiler le modèle
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Afficher le résumé du modèle
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True)


In [ ]:

# Entraîner le modèle sur les données d'entraînement
history = model.fit(
    train_dataset,
    epochs=10,  # Vous pouvez ajuster le nombre d'époques en fonction de vos besoins
    validation_data=val_dataset
)

In [ ]:

# Évaluer le modèle sur les données de validation
val_loss = model.evaluate(val_dataset)
print(f'Validation Loss: {val_loss}')

# (Optionnel) Évaluer le modèle sur les données de test
test_loss = model.evaluate(test_dataset)
print(f'Test Loss: {test_loss}')

NameError: name 'model' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Faire des prédictions sur les données de test
test_predictions = model.predict(test_dataset)

# Extraire les cibles réelles des données de test
test_targets = np.concatenate([y for x, y in test_dataset], axis=0)

# Inverser la normalisation pour obtenir les valeurs réelles
test_predictions = test_predictions * std[1] + mean[1]
test_targets = test_targets * std[1] + mean[1]

# Tracer les valeurs réelles et prédites
plt.figure(figsize=(14, 7))
plt.plot(test_targets[:30], label='Valeurs Réelles')
plt.plot(test_predictions[:30], label='Valeurs Prédites')
plt.xlabel('Index')
plt.ylabel('Température (°C)')
plt.title('Valeurs Réelles vs. Valeurs Prédites (30 premières valeurs)')
plt.legend()
plt.show()

In [ ]:
# Prédiction de la température sur les 30 prochains jours
# Pour cela, nous utilisons la dernière séquence de test pour prédire les 30 prochains jours
last_sequence = raw_data[-sequence_length:]  # Dernière séquence
predictions_30_days = []

for _ in range(30):
    # Ajouter une nouvelle dimension pour correspondre à l'entrée attendue du modèle
    input_sequence = last_sequence[np.newaxis, ...]

    # Prédire la température
    prediction = model.predict(input_sequence)[0][0]

    # Inverser la normalisation pour obtenir la valeur réelle
    prediction = prediction * std[1] + mean[1]
    predictions_30_days.append(prediction)

    # Ajouter la nouvelle prédiction à la séquence et enlever la plus ancienne
    new_value = (prediction - mean[1]) / std[1]  # Normaliser la nouvelle prédiction
    last_sequence = np.roll(last_sequence, -1, axis=0)  # Décaler la séquence
    last_sequence[-1, 1] = new_value  # Mettre à jour avec la nouvelle prédiction

# Afficher les prédictions pour les 30 prochains jours
plt.figure(figsize=(14, 7))
plt.plot(predictions_30_days, label='Prédictions pour les 30 prochains jours')
plt.xlabel('Jour')
plt.ylabel('Température (°C)')
plt.title('Prédictions de Température pour les 30 Prochains Jours')
plt.legend()
plt.show()